In [11]:
pip install msedge.selenium_tools


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\SUHANI\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install zenrows

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\SUHANI\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [12]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\SUHANI\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [16]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\SUHANI\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [17]:
import csv
from bs4 import BeautifulSoup 
from msedge.selenium_tools import Edge, EdgeOptions
import pandas as pd
import csv
import json
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
import pandas as pd
import time
from zenrows import ZenRowsClient

In [18]:
q = input("Enter product name (Searching on flipkart)")
q = q.replace(" ","+")
test = []
names = []
flipPrices = []
prodNames = []
info,price = [],[]
client = ZenRowsClient("ee972034eff5595e3cbc074967dd4da3b2fdfbe1")
url = "https://flipkart.com/search?q="
file_name = q.replace(" ", "_")
# response recieved in bytes
resp = client.get(url+q)
print(resp.status_code)
# parsing response content using BeautifulSoup class, so that we can perform operations on it.
parsed_html = bs(resp.content, 'html.parser')
# data cleaning
raw_data = parsed_html.find("script", attrs={"id":"is_script"})
data = raw_data.contents[0].replace("window.__INITIAL_STATE__ = ","").replace(";","")
json_data = json.loads(data)
req_data = json_data["pageDataV4"]["page"]["data"]["10003"]

data_list = []
try:
    for i in range(1, len(req_data)):
        d = {}
        jd = req_data[i]["widget"]["data"]["products"]
        for j in range(len(jd)):
            jd2 = jd[j]["productInfo"]["value"]

            d["title"] = jd2["titles"]["title"]
            d["keySpecs"] = jd2["keySpecs"]
            d["rating"] = jd2["rating"]["average"]
            d["ratingCount"] = jd2["rating"]["count"]
            d["price"] = jd2["pricing"]["finalPrice"]["value"]
            d["url"] = jd2["smartUrl"]
        data_list.append(d)


except:
    pass
# dumping data to result.json file
#     print(list(data_list))
with open("flipkart"+'.json', 'w') as fp:
     json.dump(data_list, fp)

# Now let us write our data to csv file
data_file = open("flipkart"+'.csv', 'w')

# create the csv writer object
csv_writer = csv.writer(data_file)

# Counter variable used for writing
# headers to the CSV file
count = 0

for data in data_list:
    if count == 0:

        # Writing headers of CSV file
        header = data.keys()
        csv_writer.writerow(header)
        count += 1
    # Writing data of CSV file
    csv_writer.writerow(data.values())


with open('flipkart.csv') as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    rows = list(reader)
i,j = 0,2
while i in range(len(rows)):
    try:
        name = rows[j][0]
        names.append(name)
        i += 1
        j += 2
    except:
            break

402


AttributeError: 'NoneType' object has no attribute 'contents'

In [19]:

with open('flipkart.csv') as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    rows = list(reader)
i,j = 0,2
while i in range(len(rows)):
    try:
        name = rows[j][0]
        names.append(name)
        i += 1
        j += 2
    except:
            break


print("Best results",len(names))
if len(names) <= 10:
    flipkart_url = "https://www.flipkart.com/search?q=" + q
    print(flipkart_url)
    uClient = client.get(flipkart_url).content
    flipkartPage = uClient
    flipkart_html = bs(flipkartPage, "html.parser")
    bigboxes = flipkart_html.find_all("a", {"class": "s1Q9rs"})
    soup = BeautifulSoup(flipkartPage, 'html.parser')
    info = soup.select("[class~=s1Q9rs]")
    flipPrices = soup.select("[class =_30jeq3]")
    prodNames = [i.get('title') for i in info]
    names = prodNames
    df = pd.DataFrame(list(zip(prodNames, flipPrices)),
                columns =['product_name', 'Flipkart_price'])
    df.to_csv('flipkart.csv')
    print(df)
else:
    with open('flipkart.csv') as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        rows = list(reader)
    i,j = 0,2
    while i in range(len(rows)):
        try:
            price = rows[j][4]
            flipPrices.append(price)
            i += 1
            j += 2
        except:
            break
    df = pd.DataFrame(list(zip(names, flipPrices)),
            columns =['Product_name', 'Flipkart_price'])

    print(df)
    df.to_csv('flipkart.csv')

data_file.close()

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

flipkart=''
ebay=''
amazon=''
olx=''

amazonlist = []
amazonName = []
i = 0
while i in range(len(names)):
    print(names[i])
    def amazon(name):
            try:
                global amazon
                name = " ".join(name.split(' ')[0:2])
                name1 = name.replace(" ","-")
                name2 = name.replace(" ","+")
                amazon=f'https://www.amazon.in/{name1}/s?k={name2}'
                res = client.get(f'https://www.amazon.in/{name1}/s?k={name2}',headers=headers)
                print("\nSearching in amazon:")
                soup = BeautifulSoup(res.text,'html.parser')
                amazon_page = soup.select('.a-color-base.a-text-normal')
                amazon_page_length = int(len(amazon_page))
                for i in range(0,amazon_page_length):
                    name = name.upper()
                    amazon_name = soup.select('.a-color-base.a-text-normal')[i].getText().strip().upper()
                    if name in amazon_name[0:20]:
                        amazon_name = soup.select('.a-color-base.a-text-normal')[i].getText().strip().upper()
                        amazon_price = soup.select('.a-price-whole')[i].getText().strip().upper()
                        amazonlist.append(amazon_price)
                        print("Amazon:")
                        print(amazon_name)
                        amazonName.append(amazon_name)
                        print("₹"+amazon_price)
                        print("-----------------------")
                        break
                    else:
                        i+=1
                        i=int(i)
                        if i==amazon_page_length:
                            print("amazon : No product found!")
                            print("-----------------------")
                            amazon_price = '0'
                            amazonlist.append(amazon_price)
                            amazonName.append("No similar product")
                            break

                return amazon_price
            except:
                print("amazon: No product found!")
                print("-----------------------")
                amazon_price = '0'
                amazonlist.append(amazon_price)
                amazonName.append("No similar product")
            return amazon_price
    amazon_price = amazon(names[i])
    flipkart=''
    ebay=''
    croma=''
    amazon=''
    olx=''
    i += 1

flip = flipPrices
idk = []
for i in range(len(flip)):
#     x =
    try:
        x = flip[i].text.replace('₹','')
        print(x)
        idk.append(x)
    except:
        idk = test
df = pd.DataFrame(list(zip(names,idk,amazonName, amazonlist)),
            columns =["Product_name_Flipkart","Flipkart_price",'Product_name_Amazon', 'Amazon_price'])
print(df)
df.to_csv('flipkartandamazon.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'flipkart.csv'

In [ ]:
df = pd.DataFrame(list(zip(amazonName, amazonlist)),
               columns =['Product_name', 'Amazon_price'])

print(df)

                                         Product_name Amazon_price
0                                  No similar product            0
1                                  No similar product            0
2                                  No similar product            0
3                                  No similar product            0
4                                  No similar product            0
5            NATURE’S QUEST- A DANGEROUS ENTANGLEMENT            0
6                                  No similar product            0
7            NATURE’S QUEST- A DANGEROUS ENTANGLEMENT            0
8                                  No similar product            0
9                                  No similar product            0
10                                 No similar product            0
11  THE MILLET COMPANY UNPOLISHED MILLET COMBO PAC...          491
12                                 No similar product            0
13                                 No similar product         

In [ ]:
test = flipPrices

In [ ]:
flip = flipPrices

In [ ]:
idk = []

In [ ]:
flip = flipPrices
idk = []
for i in range(len(flip)):
#     x =
    try:
        x = flip[i].text.replace('₹','')
        print(x)
        idk.append(x)
    except:
        idk = test

499
177
179
299
189
669
149
239
209
539
239
190
179
439
165
975
199
125
212
599
680
219
149
194
499
449
575
1,889
869
389
979
159
699
140
389
149
342
299
901
215


In [ ]:
import csv
# opening the csv file in 'w+' mode
file = open('flipkartandamazon.csv', 'w+', newline ='')
# writing the data into the file
with file:
    write = csv.writer(file)
    write.writerows(names)
    write.writerows(idk)
    write.writerows(amazonName)
    write.writerows(amazonlist)

In [ ]:
df = pd.DataFrame(list(zip(names,idk,amazonName, amazonlist)),
               columns =["Product_name_Flipkart","Flipkart_price",'Product_name_Amazon', 'Amazon_price'])

df

,Product_name_Flipkart,Flipkart_price,Product_name_Amazon,Amazon_price
0,Farmbean Whole Yellow Jowar Grains 2 KG / Sorg...,499,No similar product,0
1,Manna Unpolished Jowar & Bajra Millets - Sorgh...,177,No similar product,0
2,VGBNP Sorghum Daliya | Jowar Dalia | A Healthy...,179,No similar product,0
3,Manna Jowar & Bajra Millets - Sorghum & Pearl ...,299,No similar product,0
4,Annprash Best Quality Jowar whole / Jowar Sabu...,189,No similar product,0
5,Nature’s Quest White Jowar 4500Gms/Sorghum/Jon...,669,NATURE’S QUEST- A DANGEROUS ENTANGLEMENT,0
6,"PRODUCER Sorghum Millet Whole Grain, Jowar Sab...",149,No similar product,0
7,Nature’s Quest White Jowar 900Gms/Sorghum/Jonn...,239,NATURE’S QUEST- A DANGEROUS ENTANGLEMENT,0
8,Manna Jowar Millet Jowar,209,No similar product,0
9,Farmbean COMBO Of 2 (1Kg Each) Varagu Rice Kod...,539,No similar product,0


In [ ]:
df.to_csv('flipkartandamazon.csv')